# Embeddings Visualization with Fashion MNIST

## What is an Embedding?

Embedding is a way to map objects (images, words, etc.) to high dimensional vectors.

## What is a Projection?

A projection is a method for taking these high dimensional vections and project them into a lower dimensional space. 

There are 2 main projections in TensorBoard
- <b>PCA</b> (Principal Component Analysis) which tries to find the top 10 principle components of the data
- <b>t-SNE</b> (T-distributed Stochastic Neighbor Embedding) which tries to preserve local structure, but will distort global structure

Read more about embeddings at the TensorFlow site
https://www.tensorflow.org/programmers_guide/embedding

# How to do a basic Embedding with Fashion MNIST?

This code is based on the Pinch of Intelligence Blog 
- http://www.pinchofintelligence.com/simple-introduction-to-tensorboard-embedding-visualisation//

## Basic workflow

- Read the Fahsion MNIST data and create an X (image) and Y (label) batch
- Create a Summary Writer
- Create the embedding Tensor from X
- Configure the projector
- Run the TF session and create a model check-point
- create the sprite image
- create the metadata (labels) file

In [18]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from tensorflow.examples.tutorials.mnist import input_data

In [19]:
# import the data and split into X and Y

test_data = np.array(pd.read_csv(r'data\fashion-mnist_test.csv'), dtype='float32')

embed_count = 1600
x_test = test_data[:embed_count, 1:] / 255
y_test = test_data[:embed_count, 0]

logdir = r'C:\Users\Fashion_MNIST\logdir'  # you will need to change this!!!

In [20]:
# setup the write and embedding tensor

summary_writer = tf.summary.FileWriter(logdir)

embedding_var = tf.Variable(x_test, name='fmnist_embedding')

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

embedding.metadata_path = os.path.join(logdir, 'metadata.tsv')
embedding.sprite.image_path = os.path.join(logdir, 'sprite.png')
embedding.sprite.single_image_dim.extend([28, 28])

projector.visualize_embeddings(summary_writer, config)

In [22]:
# run the sesion to create the model check point

with tf.Session() as sesh:
    sesh.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.save(sesh, os.path.join(logdir, 'model.ckpt'))

In [23]:
# create the sprite image and the metadata file

rows = 28
cols = 28

label = ['t_shirt', 'trouser', 'pullover', 'dress', 'coat',
          'sandal', 'shirt', 'sneaker', 'bag', 'ankle_boot']

sprite_dim = int(np.sqrt(x_test.shape[0]))

sprite_image = np.ones((cols * sprite_dim, rows * sprite_dim))

index = 0
labels = []
for i in range(sprite_dim):
    for j in range(sprite_dim):
        
        labels.append(label[int(y_test[index])])
        
        sprite_image[
            i * cols: (i + 1) * cols,
            j * rows: (j + 1) * rows
        ] = x_test[index].reshape(28, 28) * -1 + 1
        
        index += 1
        
with open(embedding.metadata_path, 'w') as meta:
    meta.write('Index\tLabel\n')
    for index, label in enumerate(labels):
        meta.write('{}\t{}\n'.format(index, label))
        
plt.imsave(embedding.sprite.image_path, sprite_image, cmap='gray')
plt.imshow(sprite_image, cmap='gray')
plt.show()

In [ ]:
# now just run tensorboard and specify the logdir path